# TODO:
- Clean (see other notebook)
- Create "distance from Seattle" feature

In [186]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.stats as st
# import pymc3 as pm
import seaborn as sns

# enables inline plots, without it plots don't show up in the notebook
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
# %config InlineBackend.figure_format = 'png'
# mpl.rcParams['figure.dpi']= 300

In [187]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 25)
pd.set_option('display.precision', 3)

In [188]:
import pickle
def load_pd_pkl(file):
    import pandas as pd
    try:
        with open(f"{file}.pkl",'rb') as picklefile:
            return pickle.load(picklefile)
    except FileNotFoundError:
        df = pd.read_csv(f"{file}.csv")
        with open(f"{file}.pkl", 'wb') as picklefile:
            pickle.dump(df, picklefile)
        return df

In [189]:
df = load_pd_pkl('data/raw_wta_df')

In [13]:
#df.to_csv('data/raw_wta_df.csv')

In [190]:
df.shape

(3555, 31)

In [191]:
df.reset_index(drop=True, inplace=True)

In [192]:
df['votes'] = df['votes'].astype(int)
df['countreports'] = df['countreports'].astype(int)
df['rating'] = df['rating'].astype(float)
df['length'] = df['length'].astype(float)
df['gain'] = df['gain'].astype(float)
df['hpoint'] = df['hpoint'].astype(float)
df['lat'] = df['lat'].astype(float)
df['long'] = df['long'].astype(float)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3555 entries, 0 to 3554
Data columns (total 31 columns):
name                     3555 non-null object
region                   3555 non-null object
subregion                2942 non-null object
votes                    3555 non-null int64
rating                   3555 non-null float64
length                   2361 non-null float64
lengthtype               2219 non-null object
gain                     1854 non-null float64
hpoint                   1946 non-null float64
fee                      2013 non-null object
lat                      2607 non-null float64
long                     2607 non-null float64
trailhead1               2961 non-null object
trailhead2               1081 non-null object
author1                  1291 non-null object
author2                  1291 non-null object
countreports             3555 non-null int64
Wildflowers/Meadows      3555 non-null int64
Mountain views           3555 non-null int64
Dogs allowed on l

In [198]:
# Clean hikes which faultily have the 'hpoint' assigned to the "length" to NaN
df.loc[df['length'] == df['hpoint'], 'length'] = np.nan

In [200]:
# Good! All length types exist with a length preceding it.
df[df['length'].notna() & df['length'].isna()]

,name,region,subregion,votes,rating,length,lengthtype,gain,hpoint,fee,lat,long,trailhead1,trailhead2,author1,author2,countreports,Wildflowers/Meadows,Mountain views,Dogs allowed on leash,Established campsites,Waterfalls,Good for kids,Fall foliage,Ridges/passes,Wildlife,Old growth,Summits,Lakes,Rivers,Coast


In [202]:
# Accurately update length type of "of" to "miles_of_trails" like on website
df[df.lengthtype == "of"]
df.loc[df['lengthtype'] == "of", 'lengthtype'] = "miles_of_trails"

In [216]:
# Clean hikes which faultily have the 'gain assigned to the "length" to NaN
df.loc[df['length'] == df['gain'], 'length'] = np.nan
df[df.length == df.gain].shape

(0, 31)

In [227]:
# rename author1 and author2
df.rename(columns={'author1': 'org_author', "author2":"author"},inplace=True)

In [238]:
d = {i: len(df[i].unique()) for i in df.columns}
d

{'name': 3466,
 'region': 12,
 'subregion': 52,
 'votes': 89,
 'rating': 191,
 'length': 279,
 'lengthtype': 4,
 'gain': 462,
 'hpoint': 891,
 'fee': 13,
 'lat': 2220,
 'long': 2269,
 'trailhead1': 2081,
 'trailhead2': 239,
 'org_author': 4,
 'author': 114,
 'countreports': 315,
 'Wildflowers/Meadows': 2,
 'Mountain views': 2,
 'Dogs allowed on leash': 2,
 'Established campsites': 2,
 'Waterfalls': 2,
 'Good for kids': 2,
 'Fall foliage': 2,
 'Ridges/passes': 2,
 'Wildlife': 2,
 'Old growth': 2,
 'Summits': 2,
 'Lakes': 2,
 'Rivers': 2,
 'Coast': 2}

In [252]:
# There are 7 passes, and also none and n/a
df.fee.unique()

array(['Northwest Forest Pass', 'None', 'National Park Pass',
       'Discover Pass', nan, 'Refuge Entrance Pass',
       'National Monument Fee', 'None, Northwest Forest Pass',
       'Oregon State Parks Day-Use',
       'National Monument Fee, Sno-Parks Permit', 'Sno-Parks Permit',
       'Discover Pass, Sno-Parks Permit',
       'Northwest Forest Pass, Sno-Parks Permit'], dtype=object)

In [263]:
testdf = df.drop(columns=['trailhead1', 'trailhead2'], inplace=True)

KeyError: "[('trailhead1', 'trailhead2')] not found in axis"

In [261]:
testdf

In [253]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3555 entries, 0 to 3554
Data columns (total 31 columns):
name                     3555 non-null object
region                   3555 non-null object
subregion                2942 non-null object
votes                    3555 non-null int64
rating                   3555 non-null float64
length                   2219 non-null float64
lengthtype               2219 non-null object
gain                     1854 non-null float64
hpoint                   1946 non-null float64
fee                      2013 non-null object
lat                      2607 non-null float64
long                     2607 non-null float64
trailhead1               2961 non-null object
trailhead2               1081 non-null object
org_author               1291 non-null object
author                   1291 non-null object
countreports             3555 non-null int64
Wildflowers/Meadows      3555 non-null int64
Mountain views           3555 non-null int64
Dogs allowed on l

In [241]:
df[df.votes > 5].shape

(918, 31)

In [246]:
df.dropna().describe()

,votes,rating,length,gain,hpoint,lat,long,countreports,Wildflowers/Meadows,Mountain views,Dogs allowed on leash,Established campsites,Waterfalls,Good for kids,Fall foliage,Ridges/passes,Wildlife,Old growth,Summits,Lakes,Rivers,Coast
count,528.000,528.000,528.000,528.000,528.000,528.000,528.000,528.000,528.000,528.000,528.000,528.000,528.000,528.000,528.000,528.000,528.000,528.000,528.000,528.000,528.00,528.000
mean,16.307,3.413,9.511,1957.773,4357.506,47.536,-121.489,141.737,0.602,0.722,0.585,0.312,0.208,0.314,0.305,0.339,0.358,0.347,0.275,0.381,0.33,0.006
std,27.752,0.950,8.811,1745.897,1972.877,0.841,1.287,224.212,0.490,0.449,0.493,0.464,0.407,0.465,0.461,0.474,0.480,0.476,0.447,0.486,0.47,0.075
min,0.000,0.000,0.500,10.000,220.000,45.560,-124.669,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.00,0.000
25%,5.000,3.000,5.000,850.000,2800.000,47.022,-121.996,19.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.00,0.000
50%,7.500,3.560,8.000,1800.000,4700.000,47.563,-121.525,57.000,1.000,1.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.00,0.000
75%,16.000,4.000,11.000,2700.000,5840.000,48.058,-120.949,169.000,1.000,1.000,1.000,1.000,0.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.00,0.000
max,286.000,5.000,117.000,26351.000,12276.000,48.987,-117.078,1697.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.00,1.000


In [222]:
df.sample(5)

,name,region,subregion,votes,rating,length,lengthtype,gain,hpoint,fee,lat,long,trailhead1,trailhead2,author1,author2,countreports,Wildflowers/Meadows,Mountain views,Dogs allowed on leash,Established campsites,Waterfalls,Good for kids,Fall foliage,Ridges/passes,Wildlife,Old growth,Summits,Lakes,Rivers,Coast
3025,Holman Creek,North Cascades,NaN,4,3.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Holman Creek (#472.1),NaN,NaN,NaN,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2456,Chiwaukum Creek-Timothy Meadows,Central Cascades,Stevens Pass - East,2,3.00,NaN,NaN,NaN,NaN,NaN,47.689,-120.739,NaN,NaN,NaN,NaN,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1622,Paradise Glacier,Mount Rainier Area,SW - Longmire/Paradise,6,3.17,10.0,roundtrip,NaN,9000.0,National Park Pass,46.786,-121.735,Mount Rainier National Park,NaN,WTA Community,Multiple authors contributed to this report,26,0,1,0,0,1,0,0,0,0,0,0,0,0,0
1410,Canyon Lake,Central Cascades,Entiat Mountains/Lake Chelan,5,2.20,12.0,roundtrip,3800.0,6400.0,NaN,48.248,-121.192,Canyon Lake (#797),"Mt. Baker-Snoqualmie National Forest, Darringt...",WTA Community,Multiple authors contributed to this report,3,1,1,0,0,0,0,0,1,0,0,0,1,0,0
1145,Big Cedar Tree - Quinault,Olympic Peninsula,Olympia,2,4.00,0.5,roundtrip,80.0,100.0,None,47.478,-123.903,Olympic National Park,NaN,NaN,NaN,4,0,0,0,0,0,1,0,0,0,1,0,0,0,0


In [223]:
df.describe()

,votes,rating,length,gain,hpoint,lat,long,countreports,Wildflowers/Meadows,Mountain views,Dogs allowed on leash,Established campsites,Waterfalls,Good for kids,Fall foliage,Ridges/passes,Wildlife,Old growth,Summits,Lakes,Rivers,Coast
count,3555.000,3555.000,2219.000,1854.000,1946.000,2607.000,2607.000,3555.000,3555.000,3555.000,3555.000,3555.000,3555.000,3555.000,3555.000,3555.000,3555.000,3555.000,3555.000,3555.000,3555.000,3555.000
mean,6.113,2.628,8.694,1734.101,3922.856,47.524,-121.447,43.214,0.288,0.372,0.328,0.119,0.087,0.267,0.151,0.138,0.256,0.158,0.153,0.190,0.178,0.043
std,14.559,1.506,27.691,1968.667,2425.265,0.856,1.375,116.838,0.453,0.483,0.469,0.324,0.282,0.442,0.358,0.345,0.436,0.365,0.360,0.392,0.382,0.203
min,0.000,0.000,0.100,9.000,5.000,35.224,-124.732,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,1.000,2.000,3.000,400.000,1782.500,46.969,-122.139,3.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
50%,4.000,3.000,6.000,1250.000,4228.500,47.544,-121.584,10.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
75%,6.000,3.750,10.000,2515.000,5774.500,48.077,-120.926,34.000,1.000,1.000,1.000,0.000,0.000,1.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000
max,294.000,5.000,1200.000,27996.000,12276.000,49.572,-113.884,1697.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000


In [224]:
df.sample()

,name,region,subregion,votes,rating,length,lengthtype,gain,hpoint,fee,lat,long,trailhead1,trailhead2,author1,author2,countreports,Wildflowers/Meadows,Mountain views,Dogs allowed on leash,Established campsites,Waterfalls,Good for kids,Fall foliage,Ridges/passes,Wildlife,Old growth,Summits,Lakes,Rivers,Coast
713,Klawatti Peak,North Cascades,NaN,0,0.0,NaN,NaN,NaN,8485.0,"None, Northwest Forest Pass",48.554,-121.104,North Cascades National Park,NaN,NaN,NaN,2,1,1,0,0,0,0,0,0,0,0,1,0,0,0


* Fees (mix ups) (Make 1 / 0 out of those)?
* What should I cut out? Votes under 5? 
  * Votes under 5?
  * Trip reports less than something?
* Feature Engineering
  * Make "distance from Seattle" feature?
  * Do a count of the binary categorical features and a weight with that?
  * Assign number to "region" or "subregion" and add "weight" to that? Different variable for each one? Like the dummy variable and binary variables I have elsewhere?
* Removing features
  * Perhaps high trip reports == high number of votes? Only use one?
  * Drop extremes (like the > 40 mile hikes)
  * Drop those without length? or gain? or hpoint? or is NaN okay for some of them?
* Thoughts on authors and organization descriptions?

In [97]:
df.columns

Index(['name', 'region', 'subregion', 'votes', 'rating', 'length',
       'lengthtype', 'gain', 'hpoint', 'fee', 'lat', 'long', 'trailhead1',
       'trailhead2', 'author1', 'author2', 'countreports',
       'Wildflowers/Meadows', 'Mountain views', 'Dogs allowed on leash',
       'Established campsites', 'Waterfalls', 'Good for kids', 'Fall foliage',
       'Ridges/passes', 'Wildlife', 'Old growth', 'Summits', 'Lakes', 'Rivers',
       'Coast'],
      dtype='object')

In [99]:
for i in df[['name','region','subregion','lengthtype','fee','trailhead1','trailhead2','author1','author2']].columns:
    print(i,"---",df[i].unique())

name --- ['Heliotrope Ridge' 'Bear Creek Mountain' 'Skyline Trail Loop' ...
 'South Coast Beach Travelway' 'Soda Springs Big Trees'
 'South Fork Slate Creek']
region --- ['North Cascades' 'South Cascades' 'Mount Rainier Area' 'Central Cascades'
 'Puget Sound and Islands' 'Snoqualmie Region' 'Olympic Peninsula'
 'Southwest Washington' 'Issaquah Alps' 'Eastern Washington'
 'Central Washington' 'Length']
subregion --- ['Mount Baker Area' 'Goat Rocks' 'SW - Longmire/Paradise' 'Blewett Pass'
 'Seattle-Tacoma Area' 'Leavenworth Area' 'North Bend Area'
 'Cle Elum Area' 'Kitsap Peninsula' 'Bellingham Area' 'San Juan Islands'
 'Hood Canal' 'White Pass/Cowlitz River Valley' 'Salmon La Sac/Teanaway'
 'Lewis River Region' 'Cougar Mountain' 'Stevens Pass - West'
 'Stevens Pass - East' 'North Cascades Highway - Hwy 20' 'Snoqualmie Pass'
 'Chinook Pass - Hwy 410' 'Mountain Loop Highway'
 'NE - Sunrise/White River' 'Olympia' 'Tiger Mountain' 'Long Beach Area'
 'Whidbey Island' 'Methow/Sawtooth' 'NW - 

In [122]:
for i in df[['name','region','subregion','lengthtype','fee','trailhead1','trailhead2','author1','author2']].columns:
    print(i,"---",len(df[i].unique()))

name --- 3466
region --- 12
subregion --- 52
lengthtype --- 4
fee --- 13
trailhead1 --- 2081
trailhead2 --- 239
author1 --- 4
author2 --- 114


In [144]:
authors = df.groupby('author2').agg(['count','mean']).sort_values(('votes','count'), ascending=False)

In [150]:
orgs = df.groupby('author1').agg(['count','mean']).sort_values(('votes','count'), ascending=False)
orgs

votes         rating        length          gain            \
                   count    mean  count   mean  count    mean count      mean   
author1                                                                         
WTA Correspondents   699  13.775    699  2.884    695  18.801   644  1604.846   
WTA Community        366   6.552    366  2.928    363  17.326   296  1968.416   
WTA Staff            226   9.500    226  3.028    226   7.376   197  1368.741   

                   hpoint             lat          long          countreports  \
                    count      mean count    mean count     mean        count   
author1                                                                         
WTA Correspondents    631  3558.599   667  47.568   667 -121.314          699   
WTA Community         290  3759.476   358  47.427   358 -121.525          366   
WTA Staff             201  3044.045   225  47.250   225 -121.894          226   

                            Wildflowers/Meadows        Mountain views         \
                       mean               count   mean          count   mean   
author1                                                                        
WTA Correspondents  113.979                 699  0.569            699  0.589   
WTA Community        51.574                 366  0.432            366  0.560   
WTA Staff            80.261                 226  0.429            226  0.624   

                   Dogs allowed on leash        Established campsites         \
                                   count   mean                 count   mean   
author1                                                                        
WTA Correspondents                   699  0.611                   699  0.219   
WTA Community                        366  0.598                   366  0.224   
WTA Staff                            226  0.593                   226  0.239   

                   Waterfalls        Good for kids        Fall foliage         \
                        count   mean         count   mean        count   mean   
author1                                                                         
WTA Correspondents        699  0.172           699  0.476          699  0.333   
WTA Community             366  0.186           366  0.448          366  0.279   
WTA Staff                 226  0.168           226  0.442          226  0.305   

                   Ridges/passes        Wildlife        Old growth         \
                           count   mean    count   mean      count   mean   
author1                                                                     
WTA Correspondents           699  0.233      699  0.466        699  0.282   
WTA Community                366  0.230      366  0.377        366  0.276   
WTA Staff                    226  0.235      226  0.465        226  0.301   

                   Summits        Lakes        Rivers        Coast         
                     count   mean count   mean  count   mean count   mean  
author1                                                                    
WTA Correspondents     699  0.215   699  0.329    699  0.290   699  0.049  
WTA Community          366  0.180   366  0.292    366  0.358   366  0.060  
WTA Staff              226  0.181   226  0.345    226  0.305   226  0.106

In [148]:
authors[authors['rating','mean'] > 3.5]

votes          rating        length          gain  \
                      count     mean  count   mean  count    mean count   
author2                                                                   
Ken Giesbers             40   14.200     40  3.581     40   7.335    40   
Linda Roe                28   24.536     28  3.510     28   5.661    27   
Paul Kriloff             24   47.708     24  3.797     24  10.279    24   
Mason White              11   19.364     11  3.741     11  10.145    11   
Nate and Jer Barnes      11   15.455     11  3.705     11   6.409    11   
Mike Scavezze            11   19.818     11  4.155     11   7.855     9   
Melissa Ozbek             7   12.429      7  3.700      7   3.643     5   
Susan Elderkin            7    8.714      7  3.597      7   6.086     7   
Joan Burton               5    4.400      5  3.646      5   7.800     5   
Conor Dempsey             5  110.800      5  4.140      5   7.040     5   
Steven Payne              5   20.800      5  3.688      5   8.100     5   
Kim Hull                  4   39.750      4  4.050      4   7.875     4   
...                     ...      ...    ...    ...    ...     ...   ...   
Loren Drummond            2   11.500      2  3.675      2   3.600     2   
Alan Bauer                1    7.000      1  4.290      1   9.000     1   
Alex Mesick               1   17.000      1  3.530      1   6.500     0   
Walter Henze              1    1.000      1  4.000      1  24.000     1   
Ashley Morrison           1   10.000      1  3.700      1  10.000     1   
Pam and Paul Kaperick     1   55.000      1  4.380      1  11.500     1   
Lunnel Haught             1    7.000      1  3.570      1   5.500     0   
Dennis Graver             1   20.000      1  3.600      1  14.400     1   
Janelle Jones Walker      1   19.000      1  3.580      1   7.000     1   
Cindy Clark               1   15.000      1  4.070      1   8.000     1   
Cape Horn Conservancy     1   11.000      1  3.550      1   7.200     1   
Harper Meyerson           1    3.000      1  3.670      1   3.000     0   

                                hpoint             lat          long           \
                           mean  count      mean count    mean count     mean   
author2                                                                         
Ken Giesbers           1906.000     40  4816.250    40  47.226    40 -121.804   
Linda Roe               650.333     27  2002.000    28  48.164    28 -121.793   
Paul Kriloff           3724.583     24  6052.417    24  47.928    24 -121.168   
Mason White            2651.818     11  5082.545    11  47.836    11 -123.220   
Nate and Jer Barnes    1291.636     11  3715.909    11  47.311    11 -121.749   
Mike Scavezze          1777.778      9  4298.778    11  47.856    11 -121.938   
Melissa Ozbek           570.000      4  3186.000     7  47.853     7 -121.678   
Susan Elderkin          475.714      6  3633.333     7  47.287     7 -122.453   
Joan Burton            1040.000      5  4387.800     5  47.603     5 -121.842   
Conor Dempsey          2170.000      5  4103.400     5  47.554     5 -121.703   
Steven Payne           1310.000      5  3895.400     5  47.029     5 -122.147   
Kim Hull               2487.500      4  6690.250     4  47.188     4 -121.196   
...                         ...    ...       ...   ...     ...   ...      ...   
Loren Drummond          850.000      2  5553.000     2  47.344     2 -121.327   
Alan Bauer             1700.000      1  5900.000     1  46.141     1 -121.598   
Alex Mesick                 NaN      1   650.000     1  46.902     1 -123.064   
Walter Henze           3000.000      1  3930.000     1  48.903     1 -119.424   
Ashley Morrison        2300.000      1  6900.000     1  46.288     1 -121.552   
Pam and Paul Kaperick  3489.000      1  6000.000     1  47.828     1 -123.041   
Lunnel Haught               NaN      1  2300.000     1  47.417     1 -117.531   
Dennis Graver          1450.000      1  3450.000     1  47